<a href="https://cocl.us/DL0320EN_TOP_IMAGE">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0320EN/Assets/Images/Top.png" width="750" alt="IBM 10TB Storage" />
</a>

<h1>Classifying European Money Denominations: Training a Pre-trained model  </h1>

<h2>Table of Contents</h2>

<p>In this lab, you will train the pre-trained models to classify the European currency. You will use the dataset object you created in the previous lab.</p>
<ul>
    <li><a href="#gen">Create Image Dataset Generator</a></li>
    <li><a href="#ques">Questions</a>
        <ol>
            <li><a href="q31">Question 3.1: Preparation</a></li>
            <li><a href="q32">Question 3.2: Train the model</a></li>
            <li><a href="q33">Question 3.3: Plot 5 Random Images with their predictions</a></li>
            <li><a href="q34">Question 3.4: Use the second model <code>VGG16</code> to do the prediction</a></li>
        </ol>
    </li>
    <li><a href="#save">Save the trained model</a></li>
</ul>

<p>Estimated Time Needed: <b>60 min</b></p>
<hr>

<h2>Preparation</h2>

<a href="https://cocl.us/DL0320EN_storage">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0320EN/Assets/Images/ObjectStorage.png" width="750" alt="cognitive class" />
</a>

Download the datasets you needed for this lab.

In [1]:
# You can comment out this box when you already have the dataset
# Step 1: Ctrl + A : Select all
# Step 2: Ctrl + / : Comment out all; if everything selected has been comment out alreaday, then uncomment all

# Download Training Dataset
!wget --quiet -O /resources/data/train_data_keras.tar.gz https://cocl.us/DL0320EN_TRAIN_TAR_KERAS
!tar -xzf /resources/data/train_data_keras.tar.gz -C /resources/data --exclude '.*'

# Download Validation Dataset
!wget --quiet -O /resources/data/validation_data_keras.tar.gz https://cocl.us/DL0320EN_VALID_TAR_KERAS
!tar -xzf /resources/data/validation_data_keras.tar.gz -C /resources/data --exclude '.*'

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '/resources/data/train_data_keras.tar.gz'
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '/resources/data/validation_data_keras.tar.gz'


Import Keras Modules

In [2]:
# Keras Modules

import keras
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.models import Model

Using TensorFlow backend.


Import Non-Keras Modules 

In [3]:
# Non-Keras Modules

import os
from imageio import imread
from matplotlib.pyplot import imshow
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import random

<hr>

<h2 id="gen">Create Image Dataset Generator</h2>

In this section, you will need to generate the dataset by using what you learned from the previous labs.

The following code will store the paths of the image dataset.

In [4]:
# Store image dataset in the directory

train_data_dir = '/resources/data/train_data_keras'
validation_data_dir = '/resources/data/validation_data_keras'
classes = ['5', '10', '20', '50', '100', '200', '500']

Using <code>ImageDataGenerator().flow_from_directory()</code> to load the image from directory and generate the training dataset.

In [5]:
# Create the ImageDataGenerator for training dataset

train_generator = ImageDataGenerator().flow_from_directory(train_data_dir
                                                           , target_size=(224, 224)
                                                           , batch_size=10
                                                           , classes=classes
                                                           , seed=0
                                                           , shuffle=True)

Found 0 images belonging to 7 classes.


Using <code>ImageDataGenerator().flow_from_directory()</code> to load the image from directory and generate the validation dataset.

In [6]:
# Create the ImageDataGenerator for validation dataset

valid_generator = ImageDataGenerator().flow_from_directory(validation_data_dir
                                                           , target_size=(224, 224)
                                                           , batch_size=5
                                                           , classes=classes
                                                           , seed=0
                                                           , shuffle=True)

Found 0 images belonging to 7 classes.


<hr>

<h2 id="ques">Questions</h2>

<h3 id="q31">Question 3.1: Preparation</h3><b>5 point</b>

<b>Step 1</b>: Load the pre-trained model <b>ResNet50</b>. Set <code>weights='imagenet'</code>

In [7]:
# Step 1: Load the pre-trained model ResNet50

base = ResNet50()

# Type your code here

W0825 12:41:42.340509 14116 deprecation_wrapper.py:119] From C:\software\Anaconda\ide\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0825 12:41:42.947956 14116 deprecation_wrapper.py:119] From C:\software\Anaconda\ide\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 12:41:43.209720 14116 deprecation_wrapper.py:119] From C:\software\Anaconda\ide\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0825 12:41:43.320481 14116 deprecation_wrapper.py:119] From C:\software\Anaconda\ide\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0825 12:41:43.320481 14116 deprecation_wrapp

102858752/102853048 [==============================] - 9s 0us/step


<b>Step 2:</b> The following lines of code sets the attribute <code>trainable</code> to <code>False</code>. As a result, the parameters in these layers will not be affected by training.

In [8]:
# Step2: Set parameters in pre-train model to False
for layer in base.layers:
    layer.trainable = False
# Type your code here

<b>Step 3</b>: <b>ResNet50</b> is used to classify 1000 different objects; as a result, the last layer has 1000 outputs. However, you are going to classify 7 different classes, so you need to remove the last layer and replace with the new classification layer.

In [9]:
# Step 3: Replace the old classification layer with the new classification layer
sec_last_base = base.layers[-2].output
connected_model=Dense(len(classes),activation = 'softmax')(sec_last_base)
# Type your code here

In [10]:
base_input = base.input
model = Model (input = base_input,output = connected_model)


C:\software\Anaconda\ide\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Print out the model in order to show whether you get the correct answer.<br> <b>(Your peer reviewer is going to mark based on what you print here.)</b>

In [11]:
# Print the model (PLEASE DO NOT MODIFY THIS BOX)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

<h3 id="q32">Question 3.2: Train the model</h3><b>5 points</b>

<b>Step 1</b>: Use the <code>model.compile()</code> to set the configuration for the model. Use the optimizer <b>Adam</b>, loss function <b>categorical_crossentropy</b> and <code>metrics = ['accuracy']</code> as the parameters for training the model.

In [12]:
# Step 1: Use model.compile() to set the configuration


model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0825 12:41:59.686569 14116 deprecation_wrapper.py:119] From C:\software\Anaconda\ide\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



<b>Step 2: </b>Train the model with 20 epochs.

In [13]:
N_EPOCHS = 15
STEPS = train_generator.n//train_generator.batch_size 


In [14]:
# Step 2: Train the model

model.fit_generator(generator=train_generator,steps_per_epoch=STEPS, epochs=N_EPOCHS, validation_data=valid_generator)

W0825 12:42:00.027821 14116 deprecation.py:323] From C:\software\Anaconda\ide\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ValueError: `validation_steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `validation_steps` or use the `keras.utils.Sequence` class.

The information of loss and accuarcy for both training and validation is in <code>model.history.history</code>. Get the training history, and store the history into <code>train_history</code>.

In [ ]:
# Get the training history

train_history = model.history.history


<b>Step 3</b>: Plot out the result of loss for both training and validation.<br> <b>(Your peer reviewer is going to mark based on what you plot here.)</b>

In [ ]:
# Step 3: Plot the loss for both training and validation
plt.figure(1)
plt.subplot(211)
plt.plot(train_history['loss'])
plt.title('Average Loss vs Epoch')
plt.ylabel('average loss per epoch')
plt.xlabel('epoch')

plt.subplot(212)
plt.plot(train_history['val_loss'])
plt.title('Validation Loss vs Epoch')
plt.ylabel('validation loss per epoch')
plt.xlabel('epoch')
plt.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=1)

plt.show()
# Type your code here


<b>Step 4</b>: Plot out the result of accuracy for both training and validation.<br> <b>(Your peer reviewer is going to mark based on what you plot here.)</b>

In [ ]:
# Step 4: Plot the accuracy for both training and validation
plt.figure(1)
plt.subplot(211)
plt.plot(train_history['acc'])
plt.title('Accuracy vs Epoch')
plt.ylabel('Accuracy per epoch')
plt.xlabel('epoch')

plt.subplot(212)
plt.plot(train_history['val_acc'])
plt.title('Validation accuracy vs Epoch')
plt.ylabel('validation accuracy per epoch')
plt.xlabel('epoch')
plt.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=1)

plt.show()
# Type your code here


<h3 id="q33">Question 3.3: Plot 5 Random Images with their predictions</h3><b>3 points</b>

Use the <b>validation_dataset</b> to see whether model can predict. (You will use the <b>test_dataset</b> in the future lab. Use the <b>validation_dataset</b> for now.) Notice that you need to set the <code>shuffle</code> to <code>False</code>, so the the order of predictions will be the same as the file loading order.

In [ ]:
# Generate test dataset and generate the prediction results

test_valid_generator = ImageDataGenerator().flow_from_directory(validation_data_dir
                                                           , target_size=(224, 224)
                                                           , batch_size=5
                                                           , classes=classes
                                                           , seed=0
                                                           , shuffle=False)

Create a test dataset using validation data. Plot 5 random images which index is in the <code>numbers</code> list. Also print the predicted label and print a string indicate whether it has been correctly classified or mis-classified.<br> <b>(Your peer reviewer is going to mark based on what you plot here.)</b>
<p>Steps:</p>
<ol>
    <li>Use <code>MODEL.predict_generator()</code> to do the prediction</li>
    <li>Use <code>np.argmax()</code> to find the location of the maximum value</li>
    <li>Get the class labels from <code>train_generator</code> using <code>train_generator.filenames</code></li>
    <li>Get the class indices from the previous step using <code>train_generator.class_indices</code></li>
    <li>Match classes with labels like <code>[0, 1, 2, 3, 4, 5, 6] -> ['5', '10', '20', '50', '100', '200', '500']</code></li>
    <li>Get the true label by analysis folder name on the image resource path. Notice the <code>test_valid_genertor.filename</code> order is the same as the predicted order as you did not shuffle the dataset.</li>
    <li>Compare, plot and print the result</li>
</ol>

In [ ]:
# Plot five random images and their predictions

random.seed(0)
numbers = [random.randint(0, 69) for i in range(0, 5)]

# Type your code here
pred = np.argmax(model.predict_generator(test_valid_generator, 14)[numbers], axis=1)

class_labels = train_generator.class_indices
pred_labels = [list(class_labels.keys())[list(class_labels.values()).index(i)] for i in pred]
true_labels = [test_valid_generator.filenames[i].split('/')[0] for i in numbers]

ndx = 0
for i in numbers:
    result_string = 'predicted: {}'.format(pred_labels[ndx])
    if pred_labels[ndx] == true_labels[ndx]:
        result_string += ' (Correctly classified)'
    else:
        result_string += ' (Incorrectly classified)'
    plt.imshow(test_valid_generator[i // 5][0][i % 5].astype(np.uint8), aspect='auto')
    plt.show()
    print(result_string)
    ndx += 1

<h3 id="q34">Question 3.4: Use the second model <code>VGG16</code> to do the prediction</h3><b>5 points</b>

Repeat the steps in Question 3.1, 3.2 to predict the result using <code>VGG16</code> model. Then, print out the last validation accuracy.

<p>Steps:</p>
<ol>
    <li>Load the pre-trained model VGG16</li>
    <li>Replace the last classification layer with only 7 classes</li>
    <li>Set the configuration (parameters) using <code>MODEL.compile()</code></li>
    <li>Train the model</li>
    <li>Print the last validation accuracy</li>
</ol>

Hint:
<ol>
    <li>Use optimizer <code>Adam</code></li>
    <li>Use loss function <code>categorical_crossentropy</code></li>
    <li>Epoches = 5</li>
</ol>

You are welcome to try any pattern of setting and find out the best result. Please name the model variable as <code>model_vgg</code>.<br> <b>(Your peer reviewer is going to mark based on what you print here.)</b>

In [ ]:
# Use VGG16 to train the model and print out the last validation accuracy.

base1 = VGG16(weights='imagenet')

# Type your code here

In [ ]:
for layer in base1.layers:
    layer.trainable = False

In [ ]:
sec_last_base1 = base1.layers[-2].output
connected_model1=Dense(len(classes),activation = 'softmax')(sec_last_base1)
base1_input = base1.input
model_vgg = Model (input = base1_input,output = connected_model1)
model_vgg.summary()

In [ ]:
model_vgg.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_vgg.fit_generator(generator=train_generator,steps_per_epoch=STEPS, epochs=N_EPOCHS, validation_data=valid_generator)

In [ ]:
train_history_vgg = model_vgg.history.history

In [ ]:
# Plot the loss for both training and validation
plt.figure(1)
plt.subplot(211)
plt.plot(train_history_vgg['loss'])
plt.title('Average Loss vs Epoch')
plt.ylabel('average loss per epoch')
plt.xlabel('epoch')

plt.subplot(212)
plt.plot(train_history_vgg['val_loss'])
plt.title('Validation Loss vs Epoch')
plt.ylabel('validation loss per epoch')
plt.xlabel('epoch')
plt.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=1)

plt.show()
# Type your code here


In [ ]:
# Plot the accuracy for both training and validation
plt.figure(1)
plt.subplot(211)
plt.plot(train_history_vgg['acc'])
plt.title('Accuracy vs Epoch')
plt.ylabel('Accuracy per epoch')
plt.xlabel('epoch')

plt.subplot(212)
plt.plot(train_history_vgg['val_acc'])
plt.title('Validation accuracy vs Epoch')
plt.ylabel('validation accuracy per epoch')
plt.xlabel('epoch')
plt.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=1)

plt.show()


<h2 id="#save">Save the trained model</h2>

Save the trained model for the following chapters

In [ ]:
# Save the model

model.save("resnet50_keras.pt")
model_vgg.save("vgg16_keras.pt")

<a href="https://cocl.us/DLO0320EN_notebook_bott">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0320EN/Assets/Images/Bottom.png" width="750" alt="cognitive class" />
</a>

<h2>About the Authors:</h2> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

Other contributors: <a href="https://www.linkedin.com/in/michelleccarey/">Michelle Carey</a>, <a href="www.linkedin.com/in/jiahui-mavis-zhou-a4537814a">Mavis Zhou</a>, <a href="https://www.linkedin.com/in/yi-leng-yao-84451275/">Yi Leng Yao</a>

<hr>

Copyright &copy; 2018 <a href="cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu">cognitiveclass.ai</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.